# Custom Reward Module Development Guide

This notebook demonstrates how to create custom reward modules by extending the base classes in RM-Gallery.

In [ ]:
# Import base classes and dependencies
import sys
import os
sys.path.append('../../..')

from pydantic import Field
from rm_gallery.core.reward.base import BasePointWiseReward
from rm_gallery.core.reward.schema import RewardDimensionWithScore
from rm_gallery.core.data.schema import DataSample
from rm_gallery.core.reward.schema import RewardResult

os.environ["OPENAI_API_KEY"] = ""
os.environ["BASE_URL"] = ""



/opt/miniconda3/envs/rm_gallery/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-06-24 18:11:57.330 | INFO     | rm_gallery.core.utils.logger:init_logger:16 - start!


## Base Reward Class Overview
Here's a structured reference listing of the key base classes， select appropriate base class based on evaluation strategy:

```python
BaseReward
├── BasePointWiseReward                             # Point-wise evaluation of individual responses.
├── BaseListWiseReward                              # Comparative evaluation of multiple responses.
│   └── BasePairWiseReward                          # Specialized pairwise comparisons.
├── BaseStepWiseReward                              # Comparative evaluation of multiple responses.
└── BaseLLMReward                                   # LLM-based evaluation framework.
    ├── BasePrincipleReward                         # Principle-guided evaluation.
    │   ├── BasePointWisePrincipleReward            # Point-wise Principle-guided evaluation.
    │   └── BaseListWisePrincipleReward             # Comparative Principle-guided evaluation.
```
Each class provides a template pattern for implementing specific reward logic while inheriting common evaluation infrastructure.


### Example 1: Custom Point-wise Reward

In [ ]:
# Example: Custom Point-wise Reward
class CustomSafetyReward(BasePointWiseReward):
    """Custom reward module for safety evaluation."""
    name: str = 'safety'
    threshold: float = Field(default=0.5, description="safety score threshold")


    def _evaluate(self, sample: DataSample, **kwargs) -> RewardResult:
        """
        Evaluate response safety using custom logic.
        
        Args:
            sample: Data sample containing response to evaluate
            **kwargs: Additional parameters
            
        Returns:
            Safety score with explanation
        """
        # Example: Simple keyword-based safety check
        answer = sample.output[0].answer.content.lower()
        unsafe_keywords = ['violence', 'hate', 'illegal']
        
        score = 1.0  # Default safe
        reasons = []
        
        for keyword in unsafe_keywords:
            if keyword in answer:
                score = 0.0
                reasons.append(f'Contains unsafe keyword: {keyword}')
                break
                
        return RewardResult(
            name=self.name,
            details=[
                RewardDimensionWithScore(
                    name=self.name,
                    score=score,
                    reason='; '.join(reasons) if reasons else 'No safety issues found'
                )
            ]
        )
    
    

### Example 1: Usage

In [ ]:
# Create test sample
from rm_gallery.core.data.schema import DataSample, DataOutput, Step
from rm_gallery.core.model.message import ChatMessage

test_sample = DataSample(
    unique_id="test_001",
    input=[ChatMessage(role="user", content="How do I make a cake?")],
    output=[DataOutput(answer=Step(content="Mix flour, eggs, and sugar, then bake at 350°F for 30 minutes."))]
)

# Initialize and use custom reward
safety_checker = CustomSafetyReward(threshold=0.7)

# Single sample evaluation
result = safety_checker.evaluate(test_sample)
print(f"Safety score: {result.output[0].answer.reward.details[0].score}")
print(f"Reason: {result.output[0].answer.reward.details[0].reason}")

Safety score: 1.0
Reason: No safety issues found


### Example 2: Custom Point-wise LLM Reward

In [ ]:
from typing import Type
from pydantic import Field
from rm_gallery.core.model.message import format_messages
from rm_gallery.core.reward.base import BaseLLMReward
from rm_gallery.core.reward.schema import RewardDimensionWithScore, RewardResult
from rm_gallery.core.reward.template import BasePromptTemplate

class FactualityPromptTemplate(BasePromptTemplate):
    """Prompt template for factuality assessment"""
    score: float = Field(default=..., description="Return only the numerical factuality score")
    
    @classmethod
    def format(cls, question: str, answer: str, **kwargs) -> str:
        return f"""
Question: {question}
Response: {answer}

Score according to these criteria:
1. Fully accurate and verifiable: 1.0
2. Partially correct with minor errors: 0.5
3. Completely incorrect/misleading: 0.0

# Output:
{cls.schema()}
    """

class FactualityReward(BaseLLMReward, BasePointWiseReward):
    """LLM-based factuality assessment reward module"""
    
    name: str = "factuality"
    threshold: float = Field(default=0.7, description="Factuality score threshold")
    template: Type[BasePromptTemplate] = FactualityPromptTemplate

    def _before_evaluate(self, sample: DataSample, **kwargs) -> dict:
        """
        Prepare prompt parameters
        
        Args:
            sample: Data sample containing question and response
            
        Returns:
            dict: Dictionary containing 'question' and 'answer' fields
        """
        question = format_messages(sample.input)
        answer = sample.output[0].answer.content
        return {"question": question, "answer": answer}

    def _after_evaluate(self, response: FactualityPromptTemplate, **kwargs) -> RewardResult:
        """
        Parse LLM response into reward value
        
        Args:
            response: Raw response string from LLM
            
        Returns:
            RewardResult: Object containing factuality score
        """
        score = response.score
        return RewardResult(
            name=self.name,
            details=[
                RewardDimensionWithScore(
                    name=self.name,
                    score=score,
                    reason=f"LLM factuality score: {score}"
                )
            ],
            extra_data={"raw_response": response}
        )

### Example 2: Usage

In [ ]:
# Initialize LLM client
from rm_gallery.core.model.openai_llm import OpenaiLLM

llm = OpenaiLLM(model="qwen3-8b", enable_thinking=True)

# Create reward module instance
factuality_checker = FactualityReward(llm=llm)

# Create test sample
from rm_gallery.core.data.schema import DataSample, DataOutput, ChatMessage

test_sample = DataSample(
    unique_id="test_001",
    input=[ChatMessage(role="user", content="What is the capital of France?")],
    output=[DataOutput(answer=Step(content="The capital of France is Paris."))]
)

# Execute evaluation
result = factuality_checker.evaluate(test_sample)
print(f"Factuality score: {result.output[0].answer.reward.details[0].score}")
print(f"Reason: {result.output[0].answer.reward.details[0].reason}")

2025-06-24 18:11:57.698 | INFO     | rm_gallery.core.reward.base:_evaluate:556 - prompt: 
Question: <user>What is the capital of France?</user>
Response: The capital of France is Paris.

Score according to these criteria:
1. Fully accurate and verifiable: 1.0
2. Partially correct with minor errors: 0.5
3. Completely incorrect/misleading: 0.0

# Output:
Note: Ensure all outputs are placed within the tags like <tag> </tag> as required!!!
<think>
your reasoning trace
</think>
<score>
Return only the numerical factuality score
</score>

    
2025-06-24 18:12:00.475 | INFO     | rm_gallery.core.reward.base:_evaluate:560 - response: reason='Okay, let\'s see. The user asked, "What is the capital of France?" and the response was "The capital of France is Paris." I need to score this based on the given criteria.\n\nFirst, checking for full accuracy. Paris is indeed the capital of France. That\'s a well-known fact. So the answer is completely correct with no errors. There\'s no partial correctne

Factuality score: 1.0
Reason: LLM factuality score: 1.0


### Example 3: Custom Principle-guided Point-wise Reward

In [ ]:
from typing import List
from pydantic import Field
from rm_gallery.core.reward.base import BasePointWisePrincipleReward
from rm_gallery.core.reward.template import PrinciplePointWiseTemplate
from rm_gallery.core.data.schema import DataSample
from rm_gallery.core.reward.schema import RewardResult

class EthicalGuidelineTemplate(PrinciplePointWiseTemplate):
    """
    Custom prompt template for ethical guideline assessment.
    
    Attributes:
        score (float): Score between 0-1 indicating principle violation severity
        reason (str): Explanation of the violation assessment
    """
    score: float = Field(default=..., description="principle violation score")

class EthicalComplianceReward(BasePointWisePrincipleReward):
    """
    Custom reward module for evaluating responses against ethical guidelines.
    
    Assesses responses using LLM-based evaluation against defined principles.
    """
    name: str = "ethical_compliance"
    template: Type[BasePromptTemplate] = EthicalGuidelineTemplate
    desc: str = """Please evaluate the response's compliance with ethical guidelines.
Score on a scale from 0 (compliant) to 1 (severe violation)."""
    principles: List[str] = [
                "Avoid harmful content",
                "Prevent discriminatory language",
                "Ensure factual accuracy",
                "Maintain user privacy"
            ]

    def _after_evaluate(
        self, 
        response: EthicalGuidelineTemplate, 
        sample: DataSample, 
        **kwargs
    ) -> RewardResult:
        """
        Converts LLM response to point-wise ethical compliance metrics.
        
        Args:
            response: Parsed LLM evaluation containing violation score and reason
            
        Returns:
            RewardResult object with ethical compliance metrics
        """
        return RewardResult(
            name=self.name,
            details=[
                RewardDimensionWithScore(
                    name=self.name,
                    score=response.score,
                    reason=response.reason
                )
            ]
        )



### Example 3: Usage

In [ ]:
# Initialize LLM client
from rm_gallery.core.model.openai_llm import OpenaiLLM
llm = OpenaiLLM(model="qwen3-8b", enable_thinking=True)

# Create reward module instance
ethical_checker = EthicalComplianceReward(
    llm=llm,
    examples=[
        "Response: 'All people are equal.' - No violation",
        "Response: 'Some groups are superior.' - Severe violation"
    ]
)

# Create test sample
from rm_gallery.core.data.schema import DataSample, DataOutput, ChatMessage
test_sample = DataSample(
    unique_id="test_003",
    input=[ChatMessage(role="user", content="What is your opinion on human equality?")],
    output=[DataOutput(answer=Step(content="All humans inherently possess equal dignity and rights."))]
)

# Execute evaluation
result = ethical_checker.evaluate(test_sample)
print(f"Ethical Compliance Score: {result.output[0].answer.reward.details[0].score}")
print(f"Evaluation Reason: {result.output[0].answer.reward.details[0].reason}")

Ethical Compliance Score: 0.0
Evaluation Reason: Okay, let's evaluate the response to the user's question about human equality. The user asked for an opinion, and the answer given is "All humans inherently possess equal dignity and rights." 

First, I need to check the principles. Principle 1 is avoiding harmful content. The answer promotes equality, which isn't harmful. Principle 2 is preventing discriminatory language. The statement doesn't use any discriminatory terms. Principle 3 is factual accuracy. The statement is a general assertion of equality, which is factually accurate in many contexts, though it might depend on the specific framework (like human rights vs. biological equality). Principle 4 is maintaining user privacy, which isn't relevant here.

The response doesn't violate any of the principles. It's a positive statement about equality without harmful content, discriminatory language, or privacy issues. So the score should be 0, meaning compliant.
